# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

In [ ]:
def reboot_flush():
    reset_target(scope)
    target.flush()
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37
    

In [ ]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

Like with clock glitching, the scope object can set some typical glitch settings for you:

In [ ]:
if scope._is_husky:
    scope.vglitch_setup('hp', default_setup=False) # HP alone works best for Husky
else:
    scope.vglitch_setup('both', default_setup=False) # use both transistors

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
import re
import struct
gc.set_range("ext_offset", 0, 150)
g_step = 0.4
gc.set_global_step(g_step)

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

gc.set_range("width", ???, ???)
gc.set_range("offset", ???, ???)
gc.set_global_step(???)

gc.set_step("ext_offset", 1) # check each clock cycle

scope.adc.timeout = 0.1

reboot_flush()

successes = 0

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    ret = scope.capture()
    scope.io.vglitch_reset()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
        if ???:
            gc.add("reset")
        else:
            if ???: #for loop check
                successes +=1 
                gc.add("success")
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal")
                    
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

Let's see where we needed to target for our glitch to work:

In [ ]:
gc.calc(["width", "offset"], "success_rate")

In [ ]:
scope.dis()
target.dis()